In [4]:
class Box:
    
    '''
    Constructor
    '''
    # to check if a box is complete
    def __init__(self, x, y):
        # set of box coordinates
        self.coordinates = [(x, y), (x + 1, y), (x, y + 1), (x + 1, y + 1)]
        # top left coordinates
        self.top_left = (x, y)
        # top line
        self.top_line = (self.coordinates[0], self.coordinates[1])
        # right line
        self.right_line = (self.coordinates[1], self.coordinates[3])
        # bottom line
        self.bottom_line = (self.coordinates[2],  self.coordinates[3])
        # left line
        self.left_line = (self.coordinates[0],  self.coordinates[2])
        # set of box lines
        self.lines = ([self.top_line, self.right_line, self.bottom_line, self.left_line])
        
        # indicator of connected points
        self.top_connected = False
        self.right_connected = False
        self.bottom_connected = False
        self.left_connected = False

        # players who get boxes
        self.owner = None
        self.captured = False

        # set value from box
        self.value = 1

    '''
    Function to create edges or connect two points in a box
    '''
    def make_edge(self, coordinates):
        edge = coordinates
        edge_created = False
        if edge in self.lines:
            if edge == self.top_line and self.top_connected == False:
                self.top_connected = True
                edge_created = True
            elif edge == self.right_line and self.right_connected == False:
                self.right_connected = True
                edge_created = True
            elif edge == self.bottom_line and self.bottom_connected == False:
                self.bottom_connected = True
                edge_created = True
            elif edge == self.left_line and self.left_connected == False:
                self.left_connected = True
                edge_created = True
        if self.top_connected == True and self.right_connected == True and self.bottom_connected == True and self.left_connected == True:
            self.captured = True
        return edge_created

In [5]:
from collections import deque

class Board:
    '''
    Constructor
    '''
    def __init__(self, column, row):
        column = column - 1
        row = row - 1
        #initiation of player scores and ai
        self.player_score = 0
        self.ai_score = 0
        #setting width and length of board
        self.m = column
        self.n = row

        self.boxes = self.generate_boxes(column, row)
        self.open_vectors = self.generate_vectors(column, row)
        #create sets of objects that store the connected dots
        self.connected_vectors = set()
        #initialization of alpha and beta values for the alpha-beta pruning
        self.alpha = -100000
        self.beta = 100000

    '''
    Function to generate Box objects on the board.
    These boxes function to store the meta data needed for
    represents state information.
    '''
    def generate_boxes(self, m, n):
        row = m
        column = n
        boxes = [[Box(0, 0) for j in range(column)] for i in range(row)]
        for i in range(m):
            for j in range(n):
                boxes[i][j] = (Box(i, j))
        return boxes

    '''
    Function to generate vectors representing all possible motions that can be
    played on a board with size m rows and n columns. The vectors that are stored as tuples
    contain the coordinates of the point and is stored in a queue. 
    The queue and the list containing the boxes associated with the
    the coordinates of these points are used to represent the game state.
    The vector format is ((x1, y1), (x2, y2))
    '''
    def generate_vectors(self, m, n):
        vec = deque()
        for i in range(0, m + 1):
            for j in range(0, n):
                vec.append(((j, i), (j + 1, i)))
                if i < m:
                    vec.append(((j, i), (j, i + 1)))
            if i < m:
                vec.append(((n, i), (n, i + 1)))
        return vec

    '''
    Function to display the current state board on the command line.
    '''
    def display(self):
        print("Player: %s" % self.player_score)
        print("AI: %s" % self.ai_score)

        #printing x-axis labels
        h_str = "\n  "
        for i in range(self.m + 1):
            h_str = h_str + "   %s" % i
        print(h_str)

        #printing the board
        box_value = " "
        for i in range(self.m + 1):
            
            #add a y-axis label at the beginning of the line
            h_str = "%s " % i
            v_str = "     "
            
            for j in range(self.n + 1):
                
                #check horizontal connection
                if ((j - 1, i), (j, i)) in self.connected_vectors:
                    h_str = h_str + "---o"
                else:
                    h_str = h_str + "   o"
                
                #check the box value of the box
                if j < self.n:
                    if self.boxes[j][i - 1].top_left == (j, i - 1):
                        box_value = self.boxes[j][i - 1].value
                else:
                    box_value = " "

                #check vertical connection
                if ((j, i - 1), (j, i)) in self.connected_vectors:
                    v_str = v_str + "| %s " % box_value
                else:
                    v_str = v_str + "  %s " % box_value
            print(v_str)
            print(h_str)
        print("")

    '''
    Function to move on the board. Moves are checked by searching for the 
    selected coordinates in the "open_vectors" queue.
    If found, then pop those coordinates from "open_vectors" and add 
    those coordinates to "connected_vectors".
    Next, check if any of the boxes have been captured.
    If the move doesn't work, then return -1. 
    '''
    def move(self, coordinates, player):
        if player == True:
            player = 1
        elif player == False:
            player = 0
        if coordinates in self.open_vectors:
            self.open_vectors.remove(coordinates)
            self.connected_vectors.add(coordinates)
            self.check_boxes(coordinates, player)
            return 0
        else:
            return -1

    '''
    Function to receive the set of coordinates and the current player.
    In this function, an iteration of the list box is performed to find
    out whether a line is in the stored box.
    If a box is formed, the player who captures the box is given
    additional points and becomes the owner of the box.
    '''
    def check_boxes(self, coordinates, player):
        for i in range(self.m):
            for j in range(self.n):
                box = self.boxes[i][j]
                if coordinates in box.lines:
                    box.make_edge(coordinates)
                if box.captured is True and box.owner is None:
                    box.owner = player
                    self.prevComplete = True
                    # human player
                    if player == 0:
                        self.player_score += box.value
                    # ai player
                    elif player == 1:
                        self.ai_score += box.value

In [6]:
from copy import deepcopy
from tkinter import *
import numpy as np


class DotsAndBoxes:
    # Some class constants that can be changed quickly
    BOARD_SIZE = 600
    NUMBER_OF_DOTS = 4  # total amount of dots is n x n
    SYMBOL_SIZE = (BOARD_SIZE / 3 - BOARD_SIZE / 8) / 2
    SYMBOL_THICKNESS = 50
    DOT_COLOR = '#ffffff'
    PLAYER1_COLOR = '#04cf0b'
    PLAYER1_COLOR_LIGHT = '#88f28b'
    PLAYER2_COLOR = '#EE4035'
    PLAYER2_COLOR_LIGHT = '#EE7E77'
    THEME_COLOR = '#7BC043'
    NEUTRAL_COLOR = '#c2bfbe'
    DOT_WIDTH = 0.25 * BOARD_SIZE / NUMBER_OF_DOTS
    EDGE_WIDTH = 0.1 * BOARD_SIZE / NUMBER_OF_DOTS
    DISTANCE_BETWEEN_DOTS = BOARD_SIZE / NUMBER_OF_DOTS
    TREE_DEPTH = 10000
    ROW = 'row'
    COLUMN = 'col'

    def __init__(self):
        self.window = Tk()
        self.window.title('Dots_and_Boxes')
        self.canvas = Canvas(self.window, background='black', width=self.BOARD_SIZE, height=self.BOARD_SIZE)
        self.canvas.pack()
        self.window.bind('<Button-1>', self.click)

        self.player1_starts = True
        self.reset_board = False
        self.player1_turn = None
        self.board_status = None
        self.row_status = None
        self.col_status = None
        self.ply = None
        self.board = None
        self.turntext_handle = []
        self.already_marked_boxes = []

        self.refreshBoard()
        self.restartGame()

    def mainloop(self):
        self.window.mainloop()

    def restartGame(self):
        self.refreshBoard()
        self.board_status = np.zeros(shape=(self.NUMBER_OF_DOTS - 1, self.NUMBER_OF_DOTS - 1))
        self.row_status = np.zeros(shape=(self.NUMBER_OF_DOTS, self.NUMBER_OF_DOTS - 1))
        self.col_status = np.zeros(shape=(self.NUMBER_OF_DOTS - 1, self.NUMBER_OF_DOTS))
        self.ply = self.TREE_DEPTH
        self.board = Board(self.NUMBER_OF_DOTS, self.NUMBER_OF_DOTS)

        # Input from user in form of clicks
        self.player1_starts = not self.player1_starts
        self.player1_turn = not self.player1_starts
        self.reset_board = False
        self.turntext_handle = []

        self.already_marked_boxes = []
        self.showTurn()

    # UTILITY FUNCTIONS

    def isGridOccupiedUtil(self, logical_pos, row_column_status):
        row = logical_pos[0]
        column = logical_pos[1]

        occupied = True

        if row_column_status == self.ROW and self.row_status[column][row] == 0:
            occupied = False
        elif row_column_status == self.COLUMN and self.col_status[column][row] == 0:
            occupied = False

        return occupied

    def convGridToLogicalPosUtil(self, grid_pos):
        grid_pos = np.array(grid_pos)
        pos = (grid_pos - self.DISTANCE_BETWEEN_DOTS / 4) // (self.DISTANCE_BETWEEN_DOTS / 2)

        row_column_status = False
        logical_pos = []

        if pos[1] % 2 == 0 and (pos[0] - 1) % 2 == 0:
            r = int((pos[0] - 1) // 2)
            c = int(pos[1] // 2)
            logical_pos = [r, c]
            row_column_status = self.ROW
            # self.row_status[c][r]=1
        elif pos[0] % 2 == 0 and (pos[1] - 1) % 2 == 0:
            c = int((pos[1] - 1) // 2)
            r = int(pos[0] // 2)
            logical_pos = [r, c]
            row_column_status = self.COLUMN

        return logical_pos, row_column_status

    def isGameOverUtil(self):
        return (self.row_status == 1).all() and (self.col_status == 1).all()

    def markBox(self):
        boxes = np.argwhere(self.board_status == -4)
        for box in boxes:
            if list(box) not in self.already_marked_boxes and list(box) != []:
                self.already_marked_boxes.append(list(box))
                color = self.PLAYER1_COLOR_LIGHT
                self.fillBox(box, color)

        boxes = np.argwhere(self.board_status == 4)
        for box in boxes:
            if list(box) not in self.already_marked_boxes and list(box) != []:
                self.already_marked_boxes.append(list(box))
                color = self.PLAYER2_COLOR_LIGHT
                self.fillBox(box, color)

    def updateBoard(self, logical_pos, row_column_status):
        row = logical_pos[0]
        column = logical_pos[1]
        last_edge = 1

        if self.player1_turn:
            last_edge = -1

        if column < (self.NUMBER_OF_DOTS - 1) and row < (self.NUMBER_OF_DOTS - 1):
            self.board_status[column][row] = (abs(self.board_status[column][row]) + 1) * last_edge
            if abs(self.board_status[column][row]) == 4:
                self.another_turn = True

        if row_column_status == self.ROW:
            self.row_status[column][row] = 1
            if column >= 1:
                self.board_status[column - 1][row] = (abs(self.board_status[column - 1][row]) + 1) * last_edge
                if abs(self.board_status[column - 1][row]) == 4:
                    self.another_turn = True

        elif row_column_status == self.COLUMN:
            self.col_status[column][row] = 1
            if row >= 1:
                self.board_status[column][row - 1] = (abs(self.board_status[column][row - 1]) + 1) * last_edge
                if abs(self.board_status[column][row - 1]) == 4:
                    self.another_turn = True

    def makeEdge(self, logical_position, row_column_status):
        start_x = start_y = end_x = end_y = 0

        if row_column_status == self.ROW:
            start_x = self.DISTANCE_BETWEEN_DOTS / 2 + logical_position[0] * self.DISTANCE_BETWEEN_DOTS
            end_x = start_x + self.DISTANCE_BETWEEN_DOTS
            start_y = self.DISTANCE_BETWEEN_DOTS / 2 + logical_position[1] * self.DISTANCE_BETWEEN_DOTS
            end_y = start_y
        elif row_column_status == self.COLUMN:
            start_y = self.DISTANCE_BETWEEN_DOTS / 2 + logical_position[1] * self.DISTANCE_BETWEEN_DOTS
            end_y = start_y + self.DISTANCE_BETWEEN_DOTS
            start_x = self.DISTANCE_BETWEEN_DOTS / 2 + logical_position[0] * self.DISTANCE_BETWEEN_DOTS
            end_x = start_x

        if self.player1_turn:
            color = self.PLAYER1_COLOR
        else:
            color = self.PLAYER2_COLOR

        self.canvas.create_line(start_x, start_y, end_x, end_y, fill=color, width=self.EDGE_WIDTH)

    '''The function is to move from the ai player after the human player has made a move.
       This function calls minimax() to execute the alpha-beta pruning algorithm.
       The result of the alpha-beta pruning execution is used to determine the move ai player.'''
    
    
    def ai_move(self):
        # copy current state board for tree calculation
        state = deepcopy(self.board)
        open_vectors = deepcopy(self.board.open_vectors)
        # take coordinates from minimax algorithm
        coordinates = self.minimax(state, open_vectors, self.ply, True)
        #[0] index value is best value, 
        self.board.move(coordinates[1], 1)

        logical_position, valid_input = self.convVectorToLogicalPost(coordinates[1])
        self.updateBoard(logical_position, valid_input)
        self.makeEdge(logical_position, valid_input)
        self.markBox()
        self.refreshBoard()
        self.player1_turn = not self.player1_turn

    '''This function is to execute alpha-beta pruning algorithm.
    Parameters used are:
        * latest state boards
        * open_vectors - selectable vector set from the current state board
        * ply - total depth of game tree
        * max - "True" value represents AI and "False" represents human player
    Individual successor states are created in the main loop before calling the alpha-beta pruning algorithm
    recursively to explore the next generation in the game tree.'''
    
    def minimax(self, state, open_vectors, ply, max_min):
        # default value of best_move_val is -inf for Max layer and +inf for Min layer
        if max_min == True:
            best_move_val = (-1000000, None)
        else:
            best_move_val = (1000000, None)

        # if the successor has run out or the depth limit has been reached
        # then evaluate and return the value of the current state
        if ply == 0 or len(open_vectors) == 0:
            h = self.evaluation_function(state)
            return (h, None)

        # get successor
        for i in range(0, len(open_vectors)):
            # get the coordinates of the current successor state
            move = open_vectors.pop()
            # do a deep copy of the state to be explored
            state_copy = deepcopy(state)
            open_vectors_copy = deepcopy(open_vectors)
            state_copy.move(move, max_min)

            # add coordinates back to "open_vectors" to make sure
            # the next child state in the current depth can explore the rest of the state in the tree
            open_vectors.appendleft(move)

            # Alpha-Beta Pruning
            # check required values(alpha on min node and beta on max node) before exploring child nodes.
            h = self.evaluation_function(state_copy)
            if max_min == True:
                if h >= state_copy.beta:
                    return (h, move)
                else:
                    state_copy.alpha = max(state_copy.alpha, h)
            else:
                if h <= state_copy.alpha:
                    return (h, move)
                else:
                    state_copy.beta = min(state_copy.beta, h)

            # call minimax() recursively with child state
            # the goal state is propagated back up the tree at the end of the recursion, 
            # when the depth limit is reached or the open moves run out
            next_move = self.minimax(state_copy, open_vectors_copy, ply - 1, not max_min)

           # compare score of child state with "best_move_val"
            if max_min == True:
                # at max level, look for a score greater than the current maximum score
                if next_move[0] > best_move_val[0]:
                    best_move_val = (next_move[0], move)
            else:
                
                # on the min level, look for a score that is smaller than the current minimum score
                if next_move[0] < best_move_val[0]:
                    best_move_val = (next_move[0], move)
        return best_move_val
    
    '''
    An evaluation function is used to calculate the heuristic value of a given state.
    The heuristic value is calculated by subtracting the AI player's total score from the human player's total score.
    '''
    def evaluation_function(self, state):
        h = state.ai_score - state.player_score
        return h

    def displayGameOver(self):
        player1_score = len(np.argwhere(self.board_status == -4))
        player2_score = len(np.argwhere(self.board_status == 4))

        if player1_score > player2_score:
            # Player 1 wins
            text = 'Player Wins '
            color = self.PLAYER1_COLOR
        elif player2_score > player1_score:
            text = 'AI Wins '
            color = self.PLAYER2_COLOR
        else:
            text = 'Its a tie'
            color = self.NEUTRAL_COLOR

        self.canvas.delete("all")
        self.canvas.create_text(self.BOARD_SIZE / 2, self.BOARD_SIZE / 3, font="cmr 60 bold", fill=color, text=text)

        score_text = 'Scores \n'
        self.canvas.create_text(self.BOARD_SIZE / 2, 5 * self.BOARD_SIZE / 8, font="cmr 40 bold", fill=self.THEME_COLOR,
                                text=score_text)

        score_text = 'Player : ' + str(player1_score) + '\n'
        score_text += 'AI : ' + str(player2_score) + '\n'
        # score_text += 'Tie                    : ' + str(self.tie_score)
        self.canvas.create_text(self.BOARD_SIZE / 2, 3 * self.BOARD_SIZE / 4, font="cmr 30 bold", fill=self.THEME_COLOR,
                                text=score_text)
        self.reset_board = True

        score_text = 'Click to play again \n'
        self.canvas.create_text(self.BOARD_SIZE / 2, 15 * self.BOARD_SIZE / 16, font="cmr 20 bold",
                                fill=self.NEUTRAL_COLOR, text=score_text)

    def refreshBoard(self):
        for i in range(self.NUMBER_OF_DOTS):
            x = i * self.DISTANCE_BETWEEN_DOTS + self.DISTANCE_BETWEEN_DOTS / 2
            self.canvas.create_line(x, self.DISTANCE_BETWEEN_DOTS / 2, x,
                                    self.BOARD_SIZE - self.DISTANCE_BETWEEN_DOTS / 2, fill='gray', dash=(2, 2))
            self.canvas.create_line(self.DISTANCE_BETWEEN_DOTS / 2, x,
                                    self.BOARD_SIZE - self.DISTANCE_BETWEEN_DOTS / 2, x, fill='gray', dash=(2, 2))

        for i in range(self.NUMBER_OF_DOTS):
            for j in range(self.NUMBER_OF_DOTS):
                start_x = i * self.DISTANCE_BETWEEN_DOTS + self.DISTANCE_BETWEEN_DOTS / 2
                end_x = j * self.DISTANCE_BETWEEN_DOTS + self.DISTANCE_BETWEEN_DOTS / 2
                self.canvas.create_oval(start_x - self.DOT_WIDTH / 2, end_x - self.DOT_WIDTH / 2,
                                        start_x + self.DOT_WIDTH / 2, end_x + self.DOT_WIDTH / 2, fill=self.DOT_COLOR,
                                        outline=self.DOT_COLOR)

    def showTurn(self):
        text = 'Next turn: '
        if self.player1_turn:
            text += 'Player'
            color = self.PLAYER1_COLOR
        else:
            text += 'AI'
            color = self.PLAYER2_COLOR

        self.canvas.delete(self.turntext_handle)
        self.turntext_handle = self.canvas.create_text(self.BOARD_SIZE - 5 * len(text),
                                                       self.BOARD_SIZE - self.DISTANCE_BETWEEN_DOTS / 8,
                                                       font="cmr 15 bold", text=text, fill=color)

    def fillBox(self, box, color):
        start_x = self.DISTANCE_BETWEEN_DOTS / 2 + box[1] * self.DISTANCE_BETWEEN_DOTS + self.EDGE_WIDTH / 2
        start_y = self.DISTANCE_BETWEEN_DOTS / 2 + box[0] * self.DISTANCE_BETWEEN_DOTS + self.EDGE_WIDTH / 2
        end_x = start_x + self.DISTANCE_BETWEEN_DOTS - self.EDGE_WIDTH
        end_y = start_y + self.DISTANCE_BETWEEN_DOTS - self.EDGE_WIDTH
        self.canvas.create_rectangle(start_x, start_y, end_x, end_y, fill=color, outline='')

    def click(self, event):
        if not self.reset_board:
            if self.player1_turn:
                grid_position = [event.x, event.y]
                logical_position, valid_input = self.convGridToLogicalPosUtil(grid_position)
                if valid_input and not self.isGridOccupiedUtil(logical_position, valid_input):
                    integers = self.convLogicalPosToVector(logical_position, valid_input)
                    coordinates = ((integers[0], integers[1]), (integers[2], integers[3]))
                    self.board.move(coordinates, 0)
                    self.updateBoard(logical_position, valid_input)
                    self.makeEdge(logical_position, valid_input)
                    self.markBox()
                    self.refreshBoard()
                    self.player1_turn = not self.player1_turn
            else:
                self.ai_move()

            if self.isGameOverUtil():
                # self.canvas.delete("all")
                self.displayGameOver()
            else:
                self.showTurn()
        else:
            self.canvas.delete("all")
            self.restartGame()
            self.reset_board = False

    def convLogicalPosToVector(self, logical_pos, row_column_status):
        row = logical_pos[0]
        col = logical_pos[1]
        x1 = y1 = x2 = y2 = 0
        if row_column_status == self.ROW:
            x1 = row
            y1 = col
            x2 = x1 + 1
            y2 = col
        elif row_column_status == self.COLUMN:
            x1 = row
            y1 = col
            x2 = row
            y2 = y1 + 1

        return x1, y1, x2, y2

    def convVectorToLogicalPost(self, coordinates):
        row_column_status = None
        x1 = coordinates[0][0]
        y1 = coordinates[0][1]
        x2 = coordinates[1][0]
        y2 = coordinates[1][1]
        row = x1
        col = y1
        if x1 == x2:
            row_column_status = self.COLUMN
        elif y1 == y2:
            row_column_status = self.ROW

        logical_pos = [row, col]

        return logical_pos, row_column_status

game_instance = DotsAndBoxes()
game_instance.mainloop()